In [ ]:
# Version 1.0

import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.datasets import mnist
from keras.callbacks import TensorBoard, LearningRateScheduler

# Creat logs file
file_name = 'rnn_model'
tensorboard = TensorBoard(log_dir = "logs\\{}".format(file_name))

# Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize
scale = 255
x_train = x_train.astype('float32') / scale
x_test = x_test.astype('float32') / scale

# Parameter
epochs = 10
learning_rate = 0.001
decay_rate = learning_rate / epochs
batch_size = 100

# Decay the learning rate
def exp_decay(epoch):
    lrate = learning_rate * np.exp(-decay_rate * epoch)
    return lrate

lr_rate = LearningRateScheduler(exp_decay)

# Model
num_classes = 10
model = Sequential()
model.add(LSTM(128, input_shape = (x_train.shape[1:]), activation = 'relu', return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation = 'softmax'))

model.summary()

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate = learning_rate, decay = decay_rate)
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = optimizer,
             metrics = ['accuracy'])

callback_list = [tensorboard, lr_rate]

# Start training
hist = model.fit(x_train, y_train, 
                 batch_size = batch_size, epochs = epochs, 
                 verbose = 2, shuffle = True,
                 callbacks = callback_list)

print('Optimization Finished!')

# Calculate accuracy
score = model.evaluate(x_test, y_test, verbose = 0)
print('Accuracy:', score[1])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 28, 128)           80384     
                                                                 
 dropout_2 (Dropout)         (None, 28, 128)           0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 213,258
Trainable params: 213,258
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
600/600 - 57s - loss: 0.5807 - accura